In [ ]:
#!pip install dm-pybloom
#!pip install bitarray
#pip install kafka-python
#!curl https://www.mit.edu/~ecprice/wordlist.10000 -o words.txt
#!ls -l

#pip install --no 

In [115]:
from kafka3 import TopicPartition, KafkaConsumer, KafkaProducer
from time import sleep
from pybloom import ScalableBloomFilter, BloomFilter
from json import loads,dumps
from datetime import datetime
import hashlib
from bitarray import bitarray

#Global Variables
SERVER = 'BR'
TOPICOS ='Topicos'+SERVER
CAPACITY = 100
ERROR_RATE = 0.005
MODE = ScalableBloomFilter.SMALL_SET_GROWTH

delay = {
    
    'MZ_PT' : 200/1000,
    'MZ_BR' : 300/1000,
    'PT_MZ' : 200/1000,
    'PT_BR' : 250/1000,
    'BR_MZ' : 300/1000,
    'BR_PT' : 250/1000
}

In [ ]:

consumer = KafkaConsumer( bootstrap_servers='kafka:9092',  value_deserializer=lambda x: loads(x.decode('utf-8'))) 
producer = KafkaProducer(bootstrap_servers='kafka:9092' , value_serializer=lambda x:  dumps(x).encode('utf-8'))
      

def subscribe(consumer, topico):
    lista = consumer.subscription()
    if lista is None:
        consumer.subscribe(topico)
    else:
        lista.add(topico)
        consumer.subscribe(lista)
        
def Encaminhamento(aux_topico, mensagem):
    for i in range(len(List_bloofGW)):
        for aux_serverGW in List_bloofGW[i].keys():
            aux_sbfGW = List_bloofGW[i][aux_serverGW]
            if aux_topico in aux_sbfGW:
                message = {'server':aux_serverGW, 'message' : mensagem, 'from':SERVER,'topico': aux_serverGW.lower()+'_'+aux_topico}
                producer.send('Encaminhamento_'+aux_serverGW.upper(), value = message)
                
subscribe(consumer,TOPICOS)
subscribe(consumer,'central_SBF')
subscribe(consumer,'Encaminhamento_'+SERVER)
listaTopicos = {}
bloof = ScalableBloomFilter(initial_capacity=CAPACITY, error_rate=ERROR_RATE, mode=MODE)
List_bloofGW = []

print('Gateway '+SERVER)
for msg in consumer:
    if  msg.topic == TOPICOS:
        topicos = msg.value['topico']
        list_bit_array =[]
        for topico in topicos:
            subscribe(consumer,topico)
            bloof.add(topico.split("_")[1])
        for i in range(len(bloof.filters)):
            dict_bit_array = {}
            dict_bit_array['BF'] = bloof.filters[i].bitarray.tolist()
            dict_bit_array['Capacity'] = bloof.filters[i].capacity
            dict_bit_array['Error_tax'] = bloof.filters[i].error_rate
            list_bit_array.append(dict_bit_array)
        message = {'server' : SERVER, 'SBF' : dumps(list_bit_array) }
        producer.send('central_SBF',value = message)
        print('Enviou BF')
    elif  msg.topic == 'central_SBF':
        server_from = msg.value['server']
        #evita enciar para si mesmo
        if  server_from != SERVER: 
            print('recebeu BF do server '+server_from)
            bloofGW_aux={}
            bloofGW_aux[server_from] = ScalableBloomFilter(initial_capacity=CAPACITY, error_rate= ERROR_RATE, mode= MODE)
            list_bit_array = loads(msg.value['SBF'])
            for X in list_bit_array:
                bfilter = BloomFilter(capacity=X['Capacity'], error_rate=X['Error_tax'])
                bfilter.bitarray = X['BF']
                bloofGW_aux[server_from].filters.append(bfilter)
            List_bloofGW.append(bloofGW_aux) 
    elif  msg.topic == 'Encaminhamento_'+SERVER:
        sleep(delay[SERVER + '_' +msg.value['from']])
        message = {'message':msg.value['message'], 'from':msg.value['from']}
        producer.send(msg.value['topico'], value = message)
    else:
        # evita criar ciclo de encaminhamentos 
        if msg.value.get("from") is None:
            Encaminhamento(msg.topic.split('_')[1], msg.value)
        
        #      houve_alteracao = False
        #      listaTopicos[msg.topic] = datetime.now()
        #      body = msg.value['body']
        #      print ('receber msg de: ' + body)
        #      listaTopicos_aux = listaTopicos.copy()
        #      for elem in listaTopicos_aux:
        #          difference  = datetime.now() - listaTopicos[elem]
        #          if difference.seconds > 5:
        #              del listaTopicos[elem]
        #              houve_alteracao = True
        #       if houve_alteracao == True:
        #           print('Novo BF')
        #           bloof = ScalableBloomFilter(initial_capacity=100, error_rate=0.005, mode=ScalableBloomFilter.SMALL_SET_GROWTH)
        #           consumer.unsubscribe()
        #           subscribe(consumer,Topicos)
        #           subscribe(consumer,'central_SBF')  
        #           for elem in listaTopicos:
        #               subscribe(consumer,elem)
        #               bloof.add(elem)

Gateway BR


In [114]:
import sys
print(sys.getsizeof(bloof))

64
